In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import copy
import os
import torch
from PIL import Image
from PIL import Image, ImageDraw
from torch.utils.data import Dataset, Sampler
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
from torchvision import utils
import seaborn as sns; sns.set(style='whitegrid')
%matplotlib inline

In [2]:
# library which allows us to view model summary like keras/tf
!pip install torchsummary

In [3]:
labels_df = pd.read_csv('/kaggle/input/histopathologic-cancer-detection/train_labels.csv')
print(labels_df.head().to_markdown())
os.listdir('/kaggle/input/histopathologic-cancer-detection/')
labels_df.shape
# No duplicate ids found
labels_df[labels_df.duplicated(keep=False)]
labels_df['label'].value_counts()

|    | id                                       |   label |
|---:|:-----------------------------------------|--------:|
|  0 | f38a6374c348f90b587e046aac6079959adf3835 |       0 |
|  1 | c18f2d887b7ae4f6742ee445113fa1aef383ed77 |       1 |
|  2 | 755db6279dae599ebb4d39a9123cce439965282d |       0 |
|  3 | bc3f0c64fb968ff4a8bd33af6971ecae77c75e08 |       0 |
|  4 | 068aba587a4950175d04c680d38943fd488d6a9d |       0 |


label
0    130908
1     89117
Name: count, dtype: int64

In [4]:
torch.manual_seed(42) # fix random seed

class pytorch_data(Dataset):
    
    def __init__(self,data_dir,transform,data_type="train"):      
    
        # Get Image File Names
        cdm_data=os.path.join(data_dir,data_type)  # directory of files
        
        file_names = os.listdir(cdm_data) # get list of images in that directory  
        idx_choose = np.random.choice(np.arange(len(file_names)), 
                                      4000,
                                      replace=False).tolist()
        file_names_sample = [file_names[x] for x in idx_choose]
        self.full_filenames = [os.path.join(cdm_data, f) for f in file_names_sample]   # get the full path to images
        
        # Get Labels
        labels_data=os.path.join(data_dir,"train_labels.csv") 
        labels_df=pd.read_csv(labels_data)
        labels_df.set_index("id", inplace=True) # set data frame index to id
        self.labels = [labels_df.loc[filename[:-4]].values[0] for filename in file_names_sample]  # obtained labels from df
        self.transform = transform
      
    def __len__(self):
        return len(self.full_filenames) # size of dataset
      
    def __getitem__(self, idx):
        # open image, apply transforms and return with label
        image = Image.open(self.full_filenames[idx])  # Open Image with PIL
        image = self.transform(image) # Apply Specific Transformation to Image
        return image, self.labels[idx]


In [5]:
# define transformation that converts a PIL image into PyTorch tensors
import torchvision.transforms as transforms
data_transformer = transforms.Compose([transforms.ToTensor(),
                                       transforms.Resize((46,46))])

In [6]:
# Define an object of the custom dataset for the train folder.
data_dir = '/kaggle/input/histopathologic-cancer-detection/'
img_dataset = pytorch_data(data_dir, data_transformer, "train") # Histopathalogic images

In [7]:
# load an example tensor
img,label=img_dataset[10]
print(img.shape,torch.min(img),torch.max(img))

torch.Size([3, 46, 46]) tensor(0.2142) tensor(0.9954)


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [8]:
len_img=len(img_dataset)
len_train=int(0.8*len_img)
len_val=len_img-len_train

# Split Pytorch tensor
train_ts,val_ts=random_split(img_dataset,
                             [len_train,len_val]) # random split 80/20

print("train dataset size:", len(train_ts))
print("validation dataset size:", len(val_ts))

train dataset size: 3200
validation dataset size: 800


In [9]:
import plotly.express as px

def plot_img(x,y,title=None):

    npimg = x.numpy() # convert tensor to numpy array
    npimg_tr=np.transpose(npimg, (1,2,0)) # Convert to H*W*C shape
    fig = px.imshow(npimg_tr)
    fig.update_layout(template='plotly_white')
    fig.update_layout(title=title,height=300,margin={'l':10,'r':20,'b':10})
    fig.show()

In [10]:
# Define the following transformations for the training dataset
tr_transf = transforms.Compose([
#     transforms.Resize((40,40)),
    transforms.RandomHorizontalFlip(p=0.5), 
    transforms.RandomVerticalFlip(p=0.5),  
    transforms.RandomRotation(45),         
#     transforms.RandomResizedCrop(50,scale=(0.8,1.0),ratio=(1.0,1.0)),
    transforms.ToTensor()])

In [11]:
# For the validation dataset, we don't need any augmentation; simply convert images into tensors
val_transf = transforms.Compose([
    transforms.ToTensor()])

# After defining the transformations, overwrite the transform functions of train_ts, val_ts
train_ts.transform=tr_transf
val_ts.transform=val_transf

In [12]:
# The subset can also have transform attribute (if we asign)
train_ts.transform

Compose(
    RandomHorizontalFlip(p=0.5)
    RandomVerticalFlip(p=0.5)
    RandomRotation(degrees=[-45.0, 45.0], interpolation=nearest, expand=False, fill=0)
    ToTensor()
)

In [13]:
import pywt
#Sorter
class RMSESampler(Sampler[int]):
    
    def __init__(self, data: torch.utils.data.Subset) -> None:
        self.data = data
        
    def __len__(self) -> int:
        return len(self.data)
    
    def __iter__(self): # -> iter[int]:
        images = []
        for i in self.data.indices:
            images.append(self.data.dataset.__getitem__(i))
        rmse = []
        for img in images:
            img_array = img[0].numpy().copy()
            img_shape = img_array.shape
            img_resize = np.resize(np.resize(img_array, (img_shape[0] // 2, img_shape[1] // 2, img_shape[2] // 2)),
                (img_shape[0], img_shape[1], img_shape[2])
            )
            img_resize_array = np.asarray(img_resize)

            # RMSE
            # Get the RMSE between the original and the resized version
            img_rmse = np.sqrt(np.mean((img_array - img_resize_array) ** 2))
            rmse.append(img_rmse)
        rmse = np.negative(np.abs(rmse - np.mean(rmse)))
        rmse = torch.from_numpy(np.asarray(rmse))
        yield from torch.argsort(rmse).tolist()
        
class WDSampler(Sampler[int]):
    
    def __init__(self, data: torch.utils.data.Subset) -> None:
        self.data = data
        
    def __len__(self) -> int:
        return len(self.data)
    
    def __iter__(self): # -> iter[int]:
        images = []
        for i in self.data.indices:
            images.append(self.data.dataset.__getitem__(i))
        decomp = []
        for img in images:
            img_array = img[0].numpy().copy()
            wd = pywt.wavedec2(img_array, "db2", level=1)
            decomp.append(np.sum((np.square(wd[1][0]), np.square(wd[1][1]), np.square(wd[1][2]))))
#         decomp = np.abs(decomp - np.mean(decomp))
#         decomp = np.negative(np.abs(decomp - np.mean(decomp)))
#         decomp = torch.from_numpy(np.array(decomp))
        decomp = torch.from_numpy(np.negative(np.array(decomp)))
        yield from torch.argsort(decomp).tolist()
        
class RMSEBatchSampler(Sampler[list[int]]):
    
    def __init__(self, data: list[str], batch_size: int) -> None:
        self.data = data
        self.batch_size = batch_size
        
    def __len__(self) -> int:
        return (len(self.data) + self.batch_size - 1) // self.batch_size
    
    def __iter__(self):
        images = []
        for i in self.data.indices:
            images.append(self.data.dataset.__getitem__(i))
        rmse = []
        for img in images:
            img_array = img[0].numpy().copy()
            img_shape = img_array.shape
            img_resize = np.resize(np.resize(img_array, (img_shape[0] // 2, img_shape[1] // 2, img_shape[2] // 2)),
                (img_shape[0], img_shape[1], img_shape[2])
            )
            img_resize_array = np.asarray(img_resize)

            # RMSE
            # Get the RMSE between the original and the resized version
            img_rmse = np.sqrt(np.mean((img_array - img_resize_array) ** 2))
            rmse.append(img_rmse)
        rmse = np.negative(np.abs(rmse - np.mean(rmse)))
        rmse = torch.from_numpy(rmse)
        for batch in torch.chunk(torch.argsort(rmse), len(self)):
            yield batch.tolist()
            
class WDBatchSampler(Sampler[list[int]]):
    
    def __init__(self, data: list[str], batch_size: int) -> None:
        self.data = data
        self.batch_size = batch_size
        
    def __len__(self) -> int:
        return (len(self.data) + self.batch_size - 1) // self.batch_size
    
    def __iter__(self):
        images = []
        for i in self.data.indices:
            images.append(self.data.dataset.__getitem__(i))
        decomp = []
        for img in images:
            img_array = img[0].numpy().copy()
            wd = pywt.wavedec2(img_array, "db2", level=3)
            decomp.append(np.sum((np.square(wd[3][0]), np.square(wd[3][1]), np.square(wd[3][2])))) #np.square(wd[1][0]), np.square(wd[1][1]), np.square(wd[1][2])
#         decomp = np.abs(decomp - np.mean(decomp))
        decomp = np.negative(np.abs(decomp - np.mean(decomp)))
#         decomp = torch.from_numpy(np.array(decomp))
#         decomp = torch.from_numpy(np.negative(np.array(decomp)))
        for batch in torch.chunk(torch.argsort(torch.from_numpy(decomp)), len(self)):
            yield batch.tolist()

In [14]:
from torch.utils.data import DataLoader

# # Training DataLoader
# train_dl = DataLoader(train_ts, 
#                       shuffle=False,
#                       batch_sampler=WTFBatchSampler(train_ts, 16))

# Training DataLoader
train_dl = DataLoader(train_ts,
                      batch_size=16, 
                      shuffle=False)

# Validation DataLoader
val_dl = DataLoader(val_ts,
                    batch_size=16,
                    shuffle=False)

In [15]:
# check samples
for x,y in train_dl:
    print(x.shape,y)
    break

torch.Size([16, 3, 46, 46]) tensor([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0])


In [16]:
def findConv2dOutShape(hin,win,conv,pool=2):
    # get conv arguments
    kernel_size=conv.kernel_size
    stride=conv.stride
    padding=conv.padding
    dilation=conv.dilation

    hout=np.floor((hin+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
    wout=np.floor((win+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)

    if pool:
        hout/=pool
        wout/=pool
    return int(hout),int(wout)

import torch.nn as nn
import torch.nn.functional as F

# Neural Network
class Network(nn.Module):
    
    # Network Initialisation
    def __init__(self, params):
        
        super(Network, self).__init__()
    
        Cin,Hin,Win=params["shape_in"]
        init_f=params["initial_filters"] 
        num_fc1=params["num_fc1"]  
        num_classes=params["num_classes"] 
        self.dropout_rate=params["dropout_rate"] 
        
        # Convolution Layers
        self.conv1 = nn.Conv2d(Cin, init_f, kernel_size=3)
        h,w=findConv2dOutShape(Hin,Win,self.conv1)
        self.conv2 = nn.Conv2d(init_f, 2*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv2)
        self.conv3 = nn.Conv2d(2*init_f, 4*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv3)
        self.conv4 = nn.Conv2d(4*init_f, 8*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv4)
        
        # compute the flatten size
        self.num_flatten=h*w*8*init_f
        self.fc1 = nn.Linear(self.num_flatten, num_fc1)
        self.fc2 = nn.Linear(num_fc1, num_classes)

    def forward(self,X):
        
        # Convolution & Pool Layers
        X = F.relu(self.conv1(X)); 
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv4(X))
        X = F.max_pool2d(X, 2, 2)

        X = X.view(-1, self.num_flatten)
        
        X = F.relu(self.fc1(X))
        X=F.dropout(X, self.dropout_rate)
        X = self.fc2(X)
        return F.log_softmax(X, dim=1)

In [17]:
# Neural Network Predefined Parameters
params_model={
        "shape_in": (3,46,46), 
        "initial_filters": 8,    
        "num_fc1": 100,
        "dropout_rate": 0.25,
        "num_classes": 2}

# Create instantiation of Network class
cnn_model = Network(params_model)

# define computation hardware approach (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = cnn_model.to(device)

In [18]:
from torchsummary import summary
summary(cnn_model, input_size=(3, 46, 46),device=device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 44, 44]             224
            Conv2d-2           [-1, 16, 20, 20]           1,168
            Conv2d-3             [-1, 32, 8, 8]           4,640
            Conv2d-4             [-1, 64, 2, 2]          18,496
            Linear-5                  [-1, 100]           6,500
            Linear-6                    [-1, 2]             202
Total params: 31,230
Trainable params: 31,230
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 0.19
Params size (MB): 0.12
Estimated Total Size (MB): 0.33
----------------------------------------------------------------


In [19]:
loss_func = nn.NLLLoss(reduction="sum")

In [20]:
from torch import optim
opt = optim.Adam(cnn_model.parameters(), lr=3e-4)
lr_scheduler = ReduceLROnPlateau(opt, mode='min',factor=0.5, patience=20,verbose=1)

In [21]:
''' Helper Functions'''

        
# Function to get the learning rate
def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group['lr']

# Function to compute the loss value per batch of data
def loss_batch(loss_func, output, target, opt=None):
    
    loss = loss_func(output, target) # get loss
    pred = output.argmax(dim=1, keepdim=True) # Get Output Class
    metric_b=pred.eq(target.view_as(pred)).sum().item() # get performance metric
    
    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step()

    return loss.item(), metric_b

# Compute the loss value & performance metric for the entire dataset (epoch)
def loss_epoch(model,loss_func,dataset_dl,opt=None):
    
    run_loss=0.0 
    t_metric=0.0
    len_data=len(dataset_dl.dataset)

    loss_stats = []
    
    # internal loop over dataset
    for xb, yb in dataset_dl:
        # move batch to device
        xb=xb.to(device)
        yb=yb.to(device)
        output=model(xb) # get model output
        loss_b,metric_b=loss_batch(loss_func, output, yb, opt) # get loss per batch
        run_loss+=loss_b        # update running loss
            
        if metric_b is not None: # update running metric
            t_metric+=metric_b    
    
    loss=run_loss/float(len_data)  # average loss value
    metric=t_metric/float(len_data) # average metric value
    
    return loss, metric

In [22]:
from tqdm.notebook import trange, tqdm

def train_val(model, params,verbose=False):
    
    # Get the parameters
    epochs=params["epochs"]
    loss_func=params["f_loss"]
    opt=params["optimiser"]
    train_dl=params["train"]
    val_dl=params["val"]
    lr_scheduler=params["lr_change"]
    weight_path=params["weight_path"]
    
    loss_history={"train": [],"val": []} # history of loss values in each epoch
    metric_history={"train": [],"val": []} # histroy of metric values in each epoch
    best_model_wts = copy.deepcopy(model.state_dict()) # a deep copy of weights for the best performing model
    best_loss=float('inf') # initialize best loss to a large value
    
    ''' Train Model n_epochs '''
    
    for epoch in tqdm(range(epochs)):
        
        ''' Get the Learning Rate '''
        current_lr=get_lr(opt)
        if(verbose):
            print('Epoch {}/{}, current lr={}'.format(epoch, epochs - 1, current_lr))
        
        '''
        
        Train Model Process
        
        '''
        
        model.train()
        train_loss, train_metric = loss_epoch(model,loss_func,train_dl,opt)

        # collect losses
        loss_history["train"].append(train_loss)
        metric_history["train"].append(train_metric)
        
        '''
        
        Evaluate Model Process
        
        '''
        
        model.eval()
        with torch.no_grad():
            val_loss, val_metric = loss_epoch(model,loss_func,val_dl)
        
        # store best model
        if(val_loss < best_loss):
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            
            # store weights into a local file
            torch.save(model.state_dict(), weight_path)
            if(verbose):
                print("Copied best model weights!")
        
        # collect loss and metric for validation dataset
        loss_history["val"].append(val_loss)
        metric_history["val"].append(val_metric)
        
        # learning rate schedule
        lr_scheduler.step(val_loss)
        if current_lr != get_lr(opt):
            if(verbose):
                print("Loading best model weights!")
                model.load_state_dict(best_model_wts) 

        if(verbose):
            print(f"train loss: {train_loss:.6f}, dev loss: {val_loss:.6f}, accuracy: {100*val_metric:.2f}")
            print("-"*10) 

    # load best model weights
    model.load_state_dict(best_model_wts)
        
    return model, loss_history, metric_history

In [23]:
# params_train={
#  "train": train_dl,"val": val_dl,
#  "epochs": 50,
#  "optimiser": optim.Adam(cnn_model.parameters(),lr=3e-4),
#  "lr_change": ReduceLROnPlateau(opt,
#                                 mode='min',
#                                 factor=0.5,
#                                 patience=20,
#                                 verbose=1),
#  "f_loss": nn.NLLLoss(reduction="sum"),
#  "weight_path": "weights.pt",
# }

# ''' Actual Train / Evaluation of CNN Model '''
# # train and validate the model

# cnn_model,loss_hist,metric_hist=train_val(cnn_model,params_train, True)

In [24]:
# import seaborn as sns; sns.set(style='whitegrid')

# epochs=params_train["epochs"]

# fig,ax = plt.subplots(1,2,figsize=(12,5))

# sns.lineplot(x=[*range(1,epochs+1)],y=loss_hist["train"],ax=ax[0],label='loss_hist["train"]')
# sns.lineplot(x=[*range(1,epochs+1)],y=loss_hist["val"],ax=ax[0],label='loss_hist["val"]')
# sns.lineplot(x=[*range(1,epochs+1)],y=metric_hist["train"],ax=ax[1],label='metric_hist["train"]')
# sns.lineplot(x=[*range(1,epochs+1)],y=metric_hist["val"],ax=ax[1],label='metric_hist["val"]')
# plt.title('Convergence History')

In [25]:
import warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

In [26]:
train_dataset_mean = [[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]]
train_dataset_std = [[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]]

try:
    cnn_model.load_state_dict(reset_weights)
except:
    reset_weights = copy.deepcopy(cnn_model.state_dict())
    
# sortings
def decCalcMiddle(decomp, mean):
    return np.abs(np.subtract(decomp, mean))
def decCalcMiddleReversed(decomp, mean):
    return np.negative(np.abs(np.subtract(decomp, mean)))
def decCalcOrder(decomp, mean):
    return np.array(decomp)
def decCalcOrderReversed(decomp, mean):
    return np.negative(np.array(decomp))

sortings = [decCalcMiddle, decCalcOrder] # [decCalcMiddle, decCalcMiddleReversed, decCalcOrder, decCalcOrderReversed]
sortnames = ["from Middle", "in Order"] #["from Middle", "from Middle, reversed", "in Order", "in Reverse Order"]

# sums

def sumOfLevels(wd, level, all_levels = False):
    if level == 0:
        return np.sum((np.square(wd)))
    if all_levels:
        sum_of_levels = 0
        for l in range(level): #weird subtraction and adding here so that the approx doesn't get messed up
            sum_of_levels+=np.sum((np.square(wd[l])))
        return sum_of_levels
    else:
        return np.sum((np.square(wd[level])))

def horizontal(wd, level, all_levels = False):
    if level == 0:
        return np.sum((np.square(wd)))
    if all_levels:
        sum_of_levels = 0
        for l in range(level-1):
            if l == 0:
                sum_of_levels+=np.square(wd[l])
                continue
            sum_of_levels+=np.square(wd[l][0])
        return sum_of_levels
    else:
        return np.sum((np.square(wd[level][0])))
    
def vertical(wd, level, all_levels = False):
    if level == 0:
        return np.sum((np.square(wd)))
    if all_levels:
        sum_of_levels = 0
        for l in range(level-1):
            if l == 0:
                sum_of_levels+=np.square(wd[l])
                continue
            sum_of_levels+=np.square(wd[l][1])
        return sum_of_levels
    else:
        return np.sum((np.square(wd[level][1])))
    
def diagonal(wd, level, all_levels = False):
    if level == 0:
        return np.sum((np.square(wd)))
    if all_levels:
        sum_of_levels = 0
        for l in range(level-1):
            if l == 0:
                sum_of_levels+=np.square(wd[l])
                continue
            sum_of_levels+=np.square(wd[l][2])
        return sum_of_levels
    else:
        return np.sum((np.square(wd[level][2])))

sums = [horizontal, vertical, diagonal]#[sumOfLevels, horizontal, vertical, diagonal]
sumnames = ["Horizontal", "Vertical", "Diagonal"] #["All directions", "Horizontal", "Vertical", "Diagonal"]

reset_weights = copy.deepcopy(cnn_model.state_dict())

img_stats = []



for img_array in train_dl:
    for img in img_array[0]:
        img_stats.append(pywt.wavedec2(img, "db2", level=3))

for lev in range(1,4):
    for direction in range(0,3):
        train_dataset_mean[lev-1][direction] = np.round(np.mean(img_stats[:][lev][direction]), 2)
        train_dataset_std[lev-1][direction] = np.round(np.std(img_stats[:][lev][direction]),2)
    print(f"""TRAINING DATASET STATS FOR LEVEL {lev}:\n
    HORIZONTAL: Mean: {train_dataset_mean[lev-1][0]}\tSt.d.: {train_dataset_std[lev-1][0]}\n
    VERTICAL:   Mean: {train_dataset_mean[lev-1][1]}\tSt.d.: {train_dataset_std[lev-1][1]}\n
    DIAGONAL:   Mean: {train_dataset_mean[lev-1][2]}\tSt.d.: {train_dataset_std[lev-1][2]}\n""")
            
for lev in tqdm(range(1,4)):
    print("|||||||||||||||||||||||||||||||||||||||||||||||||||||||||")
    print(f"LEVEL {lev}")
    for sort, sortname in tqdm(zip(sortings, sortnames)):
        print("---------------------------------------------------------")
        print(f"SORTING {sortname}")
        j=0
        for summation, sumname in tqdm(zip(sums, sumnames)):
            print(train_dataset_mean)
            mean = train_dataset_mean[lev][j] # FHIX  
            cnn_model.load_state_dict(reset_weights)
            print(".........................................................")
            print(f"SUMMATION {sumname}")

            class WDSampler(Sampler[int]):

                def __init__(self, data: torch.utils.data.Subset) -> None:
                    self.data = data

                def __len__(self) -> int:
                    return len(self.data)

                def __iter__(self): # -> iter[int]:
                    images = []
                    for i in self.data.indices:
                        images.append(self.data.dataset.__getitem__(i))
                    decomp = []
                    for img in images:
                        img_array = img[0].numpy().copy()
                        wd = pywt.wavedec2(img_array, "rbio2.2", level=lev)
                        decomp.append(summation(wd, lev))
                    decomp = sort(decomp)
                    yield from torch.argsort(torch.from_numpy(decomp)).tolist()

            class WDBatchSampler(Sampler[list[int]]):

                def __init__(self, data: list[str], batch_size: int) -> None:
                    self.data = data
                    self.batch_size = batch_size

                def __len__(self) -> int:
                    return (len(self.data) + self.batch_size - 1) // self.batch_size

                def __iter__(self):
                    images = []
                    for i in self.data.indices:
                        images.append(self.data.dataset.__getitem__(i))

                    decomp = []
                    for img in images:
                        img_array = img[0].numpy().copy()
                        wd = pywt.wavedec2(img_array, "db2", level=lev)
                        decomp.append(summation(wd, lev))
                    decomp = sort(decomp, train_dataset_mean[lev][j])
                    for batch in torch.chunk(torch.argsort(torch.from_numpy(decomp)), len(self)):
                        yield batch.tolist()

            train_dl = DataLoader(train_ts, 
                                  shuffle=False,
                                  batch_sampler=WDBatchSampler(train_ts, 16))

#                 train_dl = DataLoader(train_ts, 
#                                       shuffle=False,
#                                       batch_size=16,
#                                       sampler=WDSampler(train_ts))

#                 train_dl = DataLoader(train_ts, 
#                                       shuffle=False,
#                                       batch_size=16)

            params_train={
             "train": train_dl,"val": val_dl,
             "epochs": 10,
             "optimiser": optim.Adam(cnn_model.parameters(),lr=3e-4),
             "lr_change": ReduceLROnPlateau(opt,
                                            mode='min',
                                            factor=0.5,
                                            patience=20,
                                            verbose=1),
             "f_loss": nn.NLLLoss(reduction="sum"),
             "weight_path": "weights.pt",
            }

            ''' Actual Train / Evaluation of CNN Model '''
            # train and validate the model

            cnn_model,loss_hist,metric_hist=train_val(cnn_model,params_train)

            epochs=params_train["epochs"]

            fig,ax = plt.subplots(1,2,figsize=(12,5))

            sns.lineplot(x=[*range(1,epochs+1)],y=loss_hist["train"],ax=ax[0],label='loss_hist["train"]')
            sns.lineplot(x=[*range(1,epochs+1)],y=loss_hist["val"],ax=ax[0],label='loss_hist["val"]')
            sns.lineplot(x=[*range(1,epochs+1)],y=metric_hist["train"],ax=ax[1],label='metric_hist["train"]')
            sns.lineplot(x=[*range(1,epochs+1)],y=metric_hist["val"],ax=ax[1],label='metric_hist["val"]')
            plt.title(f"LEVEL {lev} {sortname} on {sumname}")
            j+=1

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


TRAINING DATASET STATS FOR LEVEL 1:

    HORIZONTAL: Mean: 3.049999952316284	St.d.: 0.8799999952316284

    VERTICAL:   Mean: 0.03999999910593033	St.d.: 0.36000001430511475

    DIAGONAL:   Mean: 0.009999999776482582	St.d.: 0.25

TRAINING DATASET STATS FOR LEVEL 2:

    HORIZONTAL: Mean: 4.070000171661377	St.d.: 1.2699999809265137

    VERTICAL:   Mean: 0.009999999776482582	St.d.: 0.20999999344348907

    DIAGONAL:   Mean: 0.0	St.d.: 0.14000000059604645

TRAINING DATASET STATS FOR LEVEL 3:

    HORIZONTAL: Mean: 4.28000020980835	St.d.: 0.8600000143051147

    VERTICAL:   Mean: -0.0	St.d.: 0.25999999046325684

    DIAGONAL:   Mean: 0.009999999776482582	St.d.: 0.20999999344348907



  0%|          | 0/3 [00:00<?, ?it/s]

|||||||||||||||||||||||||||||||||||||||||||||||||||||||||
LEVEL 1


0it [00:00, ?it/s]

---------------------------------------------------------
SORTING from Middle


0it [00:00, ?it/s]

[[3.05, 0.04, 0.01], [4.07, 0.01, 0.0], [4.28, -0.0, 0.01]]
.........................................................
SUMMATION Horizontal


  0%|          | 0/10 [00:00<?, ?it/s]

TypeError: loss_epoch() takes from 3 to 4 positional arguments but 5 were given

## <b>11 <span style='color:#F1A424'>|</span> Inference</b> 

- Once we have trained our model using `train_val`, we can begin to utilise it to **<span style='color:#F1A424'>make some predictions</span>**
- We have a whole dataset of **<span style='color:#F1A424'>unlabelled image</span>** data in folder test
- The unique ids of each image in the dataset are located in file `sample_submission.csv`
- Like for thr training dataset, well create a data loader, using only tensor transformation
- As we have no label data, we need a slightly modified data class

In [ ]:
class pytorchdata_test(Dataset):
    
    def __init__(self, data_dir, transform,data_type="train"):
        
        path2data = os.path.join(data_dir,data_type)
        filenames = os.listdir(path2data)
        self.full_filenames = [os.path.join(path2data, f) for f in filenames]
        
        # labels are in a csv file named train_labels.csv
        csv_filename="sample_submission.csv"
        path2csvLabels=os.path.join(data_dir,csv_filename)
        labels_df=pd.read_csv(path2csvLabels)
        
        # set data frame index to id
        labels_df.set_index("id", inplace=True)
        
        # obtain labels from data frame
        self.labels = [labels_df.loc[filename[:-4]].values[0] for filename in filenames]
        self.transform = transform       
        
    def __len__(self):
        # return size of dataset
        return len(self.full_filenames)
    
    def __getitem__(self, idx):
        # open image, apply transforms and return with label
        image = Image.open(self.full_filenames[idx]) # PIL image
        image = self.transform(image)
        return image, self.labels[idx]

#### **<span style='color:#F1A424'>CHECKS</span>**

- Confirm our best performing model has been saved in the working directory 
- Confirm the test folder contents are present

In [ ]:
!ls

In [ ]:
!ls '/kaggle/input/histopathologic-cancer-detection/test' | head -n 5

#### **<span style='color:#F1A424'>FUNCTION PARAMETER DICTIONARY</span>**

Having defined a model architecture, we can load model weights

In [ ]:
# load any model weights for the model
cnn_model.load_state_dict(torch.load('weights.pt'))

#### **<span style='color:#F1A424'> TEST FILE IDS</span>**

The submission file contains all the ids to the files that are located in the test folder

In [ ]:
# sample submission
path_sub = "/kaggle/input/histopathologic-cancer-detection/sample_submission.csv"
labels_df = pd.read_csv(path_sub)
labels_df.head()
labels_df.shape

#### **<span style='color:#F1A424'>TEST IMAGE DATASET</span>**

Like we did with the training set, lets convert and store all image data in 

In [ ]:
data_dir = '/kaggle/input/histopathologic-cancer-detection/'

data_transformer = transforms.Compose([transforms.ToTensor(),
                                       transforms.Resize((46,46))])

img_dataset_test = pytorchdata_test(data_dir,data_transformer,data_type="test")
print(len(img_dataset_test), 'samples found')

#### **<span style='color:#F1A424'>PREDICTION FUNCTION</span>**

For inference, we need to set the model to `torch.no_grad`



In [ ]:
def inference(model,dataset,device,num_classes=2):
    
    len_data=len(dataset)
    y_out=torch.zeros(len_data,num_classes) # initialize output tensor on CPU
    y_gt=np.zeros((len_data),dtype="uint8") # initialize ground truth on CPU
    model=model.to(device) # move model to device
    
    with torch.no_grad():
        for i in tqdm(range(len_data)):
            x,y=dataset[i]
            y_gt[i]=y
            y_out[i]=model(x.unsqueeze(0).to(device))

    return y_out.numpy(),y_gt            

In [ ]:
y_test_out,_ = inference(cnn_model,img_dataset_test, device)            

In [ ]:
# class predictions 0,1
y_test_pred=np.argmax(y_test_out,axis=1)
print(y_test_pred.shape)
print(y_test_pred[0:5])

In [ ]:
# probabilities of predicted selection
# return F.log_softmax(x, dim=1) ie.
preds = np.exp(y_test_out[:, 1])
print(preds.shape)
print(preds[0:5])